# Old power method -related functions

In [ ]:
def abs_err(v_exact, mu_exact, v_approx, mu_approx):
    err_v = LA.norm(v_exact-v_approx)
    err_mu = abs(mu_exact-mu_approx)

    return err_mu, err_v

def rel_err(v_exact, mu_exact, v_approx, mu_approx):
    err_mu, err_v = abs_err(v_exact, mu_exact, v_approx, mu_approx)
    rel_mu = err_mu / abs(mu_exact)
    rel_v = err_v / LA.norm(v_exact)
    
    return rel_mu, rel_v

def iter_diffs(mus, W_T):
    #calculate differences between consecutive iterations of power method
    mus_shifted = mus[1:]
    diffs_mu = mus_shifted-mus[:-1]
    W_T_shifted = W_T[1:,:]
    diff_vecs_W_T = W_T_shifted - W_T[:-1, :]
    diffs_W_T = np.apply_along_axis(LA.norm, 1, diff_vecs_W_T)

    return diffs_mu, diffs_W_T
    


def residual_norm(A, v, mu):
    """Compute the 2-norm of the residual vector r = Av-av, where
    A is a 2-d numpy array, v an approximated eigenvector and a is an approximation of
    corresponding eigenvalue. Can be used to evaluate the accuracy of the estimated
    eigenvector and -value."""
    r = A@v-mu*v
    norm = LA.norm(r)

    return norm

def rel_residual(A, v, mu):
    abs_res = residual_norm(A, v, mu)
    rel = abs_res/LA.norm(A@v)

    return rel

# def hotelling(A, v, mu, n_vectors, n_iterations):
#     """NEEDS FIXING IF TO BE USED
#     Function that uses Hotelling's deflation
#     method to compute the eigenvectors that
#     correspond to the second, third, ... , (n_vectors)th
#     largest eigenvalues along
#     with their corresponding eigenvector. The largest eigenvalue
#     and corresponding eigenvector need to have been computed first.
#     Returns n_vectors eigenvectors as rows of an array and the eigenvalues
#     as a list in descending order"""

#     n = A.shape[1]
#     #array for storing the eigenvectors as its rows
#     V_T = v

#     #list for storing the eigenvalues
#     mu_s = np.array(mu)

#     #array with mu*v as columns
#     V_mu = mu*v
    

#     #each new eigenvector and eigenvalue computed iteratively
#     #using the previously computed ones
#     for i in np.arange(n_vectors-1):
        
#         #initial guess
#         w0 = np.random.rand(n)
#         norm = LA.norm(w0)
#         #normalize
#         w0 = w0/norm
    
#         #power iteration
#         for j in np.arange(n_iterations):
#             #Hotelling's deflation
#             wj = A.dot(w0)-V_mu.dot(V_T.dot(w0))

#             norm = LA.norm(wj)

#             #normalize
#             w0 = wj / norm
        
#         #calculate corresponding eigenvalue
#         mu0 = rayleigh_quotient(A, w0)

#         # store the eigenvector and eigenvalues 
#         V_T = np.row_stack((V_T, w0))
#         mu_s = np.append(mu_s, mu0)
#         V_mu = np.column_stack((V_mu, mu0*w0))

#     return V_T, mu_s 



# def power_method_thresh(A, thresh, maxiter):
#     dimA = A.shape[1]

#     #initialize
#     ws = []
#     mus = []
#     diffs = []
#     w0 = np.random.rand(dimA)
#     norm = LA.norm(w0)
#     #normalize
#     w0 = w0 / norm

#     diff = 1.
#     i = 0
#     while diff > thresh and i < maxiter:
#         mui, wi = power_step(A, w0)
#         mus.append(mui)
#         ws.append(wi)
#         diff = LA.norm(wi-w0) #difference between consecutive approximations as measure of convergence
#         diffs.append(diff)
#         w0 = wi
#         i += 1

#     return mus, ws, diffs



Error threshold vs iteration nr

In [ ]:


def find_threshold_iter_1d(run_i_errs, thresholds):
    """Find the first iteration of run i (i.e. row i) of power method error matrix which below thresholds"""
    iters = []
    for thres in thresholds:
        iter = np.where(run_i_errs < thres)
        if len(iter[0]) > 0:
            iters.append(iter[0][0])
        else:
            # Value -1 if doesn't converge below threshold
            iters.append(-1)
    return iters


def find_threshold_iter(errs, thresholds):
    """Maps find_threshold_1d to the whole error array produced by powermethod run"""
    iters = np.apply_along_axis(find_threshold_iter_1d, 1, errs, thresholds = thresholds)
    return iters


thresholds = [1e-2, 1e-3, 1e-4, 1e-5]


t_mu1s = find_threshold_iter(err_mu1s, thresholds)
t_mu2s = find_threshold_iter(err_mu2s, thresholds)
t_WT1s = find_threshold_iter(err_WT1s, thresholds)
t_WT2s = find_threshold_iter(err_WT2s, thresholds)

out_mu1 = xr.DataArray(t_mu1s, dims= ["run_id", "threshold"], coords=[np.arange(10),thresholds])
out_mu1


# Old functions for simulating symmetric matrices

In [ ]:

def create_sym_mirror(n:int):
    """Create random symmetric matrix by first making random square and then mirroring
    lower triangular values to upper triangular"""
    #random square
    #uniform [0,1]
    M = np.random.rand(n,n)
    #std normal
    #M = np.random.standard_normal((n,n)) 
    #set values above diagonal to zero
    M = np.tril(M)
    i_lower = np.tril_indices(n)
    #make symmetric
    M.T[i_lower] = M[i_lower]
    return M

def create_sym_XXT(n:int):
    M = np.random.standard_normal((n,n))
    M = M@M.T
    return M

# Old plots

In [ ]:

import matplotlib.pyplot as plt

diffs_w = [diffs_w1, diffs_w2]
diffs_mu = [diffs_mu1, diffs_mu2]

fig, axs = plt.subplots(2,1, figsize = (10,8), sharex='col')
for i in range(2):
    for j in range(n_runs):
        axs[i].plot(np.arange(1, K), diffs_mu[i][j], marker = '.', linewidth = 0.8, markersize = 1.2)
    axs[i].set_ylabel(f"$\epsilon_{i+1}$")
axs[1].set_xlabel("Iteraatio")
axs[1].set_xticks(np.arange(1, K, 10))
fig.suptitle(f"")
plt.tight_layout
plt.show()